<a href="https://colab.research.google.com/github/wmFrank/VISTA-auto-driving/blob/main/VISTA_auto_driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Autonomous Driving Policies in VISTA
Autonomous control has traditionally be dominated by algorithms that explicitly decompose individual aspects of the control pipeline. For example, in autonomous driving, traditional methods work by first detecting road and lane boundaries, and then using path planning and rule-based methods to derive a control policy. Deep learning offers something very different -- the possibility of optimizing all these steps simultaneously, learning control end-to-end directly from raw sensory inputs.

**You will explore the power of deep learning to learn autonomous control policies that are trained *end-to-end, directly from raw sensory data, and entirely within a simulated world*.**

We will use the data-driven simulation engine [VISTA](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8957584&tag=1), which uses techniques in computer vision to synthesize new photorealistic trajectories and driving viewpoints, that are still consistent with the world's appearance and fall within the envelope of a real driving scene. This is a powerful approach -- we can synthesize data that is photorealistic, grounded in the real world, and then use this data for training and testing autonomous vehicle control policies within this simulator.

In this part of the lab, you will use reinforcement lerning to build a self-driving agent with a neural network-based controller trained on RGB camera data. We will train the self-driving agent for the task of lane following. Beyond this data modality and control task, VISTA also supports [different data modalities](https://arxiv.org/pdf/2111.12083.pdf) (such as LiDAR data) and [different learning tasks](https://arxiv.org/pdf/2111.12137.pdf) (such as multi-car interactions).
<p align="center">
<img src="https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab3/img/vista_overview.png" width="600"/>
</p>


You will put your agent to the test in the VISTA environment, and potentially, on board a full-scale autonomous vehicle! Specifically, as part of the MIT lab competitions, high-performing agents -- evaluated based on the maximum distance they can travel without crashing -- will have the opportunity to be put to the ***real*** test onboard a full-scale autonomous vehicle!!! 

We start by installing dependencies. This includes installing the VISTA package itself.

In [ ]:
!nvidia-smi

In [ ]:
#Install some dependencies for visualizing the agents
!apt-get install -y xvfb python-opengl x11-utils &> /dev/null
!pip install pyvirtualdisplay scikit-video ffio pyrender &> /dev/null
!pip install tensorflow_probability==0.12.0 &> /dev/null
!pip install vista &> /dev/null
!pip install --upgrade git+https://github.com/aamini/introtodeeplearning.git &> /dev/null


In [ ]:
# import libs
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

import numpy as np
import matplotlib, cv2
import matplotlib.pyplot as plt
import base64, io, os, time
import IPython, functools
import time
from tqdm import tqdm
import tensorflow_probability as tfp
import mitdeeplearning as mdl

import vista
from vista.utils import logging
logging.setLevel(logging.ERROR)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Create an environment in VISTA

Environments in VISTA are based on and built from human-collected driving *traces*. A trace is the data from a single driving run. In this case we'll be working with RGB camera data, from the viewpoint of the driver looking out at the road: the camera collects this data as the car drives around!

We will start by accessing a trace. We use that trace to instantiate an environment within VISTA. This is our `World` and defines the environment we will use for reinforcement learning. The trace itself helps to define a space for the environment; with VISTA, we can use the trace to generate new photorealistic viewpoints anywhere within that space. This provides valuable new training data as well as a robust testing environment.

The simulated environment of VISTA will serve as our training ground and testbed for reinforcement learning. We also define an `Agent` -- a car -- that will actually move around in the environmnet, and make and carry out *actions* in this world. Because this is an entirely simulated environment, our car agent will also be simulated!



In [ ]:
# Download and extract the data for vista (auto-skip if already downloaded)
!wget -nc -q --show-progress https://www.dropbox.com/s/62pao4mipyzk3xu/vista_traces.zip
print("Unzipping data...")
!unzip -o -q vista_traces.zip
print("Done downloading and unzipping data!")

trace_root = "./vista_traces"
trace_path = [
    "20210726-154641_lexus_devens_center", 
    "20210726-155941_lexus_devens_center_reverse", 
    "20210726-184624_lexus_devens_center", 
    "20210726-184956_lexus_devens_center_reverse", 
]
trace_path = [os.path.join(trace_root, p) for p in trace_path]

# Create a virtual world with VISTA, the world is defined by a series of data traces
world = vista.World(trace_path, trace_config={'road_width': 4})

# Create a car in our virtual world. The car will be able to step and take different 
#   control actions. As the car moves, its sensors will simulate any changes it environment
car = world.spawn_agent(
    config={
        'length': 5.,
        'width': 2.,
        'wheel_base': 2.78,
        'steering_ratio': 14.7,
        'lookahead_road': True
    })

# Create a camera on the car for synthesizing the sensor data that we can use to train with! 
camera = car.spawn_camera(config={'size': (200, 320)})

# Define a rendering display so we can visualize the simulated car camera stream and also 
#   get see its physical location with respect to the road in its environment. 
display = vista.Display(world, display_config={"gui_scale": 2, "vis_full_frame": False})

# Define a simple helper function that allows us to reset VISTA and the rendering display
def vista_reset():
    world.reset()
    display.reset()
vista_reset()
    

## Create an agent(a car) in VISTA

Our goal is to learn a control policy for our agent, our (hopefully) autonomous vehicle, end-to-end directly from RGB camera sensory input. As in Cartpole, we need to define how our virtual agent will interact with its environment.

### Define agent's action functions
In the case of driving, the car agent can act -- taking a step in the VISTA environment -- according to a given control command. This amounts to moving with a desired speed and a desired *curvature*, which reflects the car's turn radius. Curvature has units $\frac{1}{meter}$. So, if a car is traversing a circle of radius $r$ meters, then it is turning with a curvature $\frac{1}{r}$. The curvature is therefore correlated with the car's steering wheel angle, which actually controls its turn radius. Let's define the car agent's step function to capture the action of moving with a desired speed and desired curvature.

In [ ]:
# First we define a step function, to allow our virtual agent to step 
# with a given control command through the environment 
# agent can act with a desired curvature (turning radius, like steering angle)
# and desired speed. if either is not provided then this step function will 
# use whatever the human executed at that time in the real data.

def vista_step(curvature=None, speed=None):
    # Arguments:
    #   curvature: curvature to step with
    #   speed: speed to step with
    if curvature is None: 
        curvature = car.trace.f_curvature(car.timestamp)
    if speed is None: 
        speed = car.trace.f_speed(car.timestamp)
    
    car.step_dynamics(action=np.array([curvature, speed]), dt=1/15.)
    car.step_sensors()

### Inspect driving trajectories in VISTA

Recall that our VISTA environment is based off an initial human-collected driving trace. Also, we defined the agent's step function to defer to what the human executed if it is not provided with a desired speed and curvature with which to move.

Thus, we can further inspect our environment by using the step function for the driving agent to move through the environment by following the human path. The stepping and iteration will take about 1 iteration per second. We will then observe the data that comes out to see the agent's traversal of the environment.

In [ ]:
import shutil, os, subprocess, cv2

# Create a simple helper class that will assist us in storing videos of the render
class VideoStream():
    def __init__(self):
        self.tmp = "./tmp"
        if os.path.exists(self.tmp) and os.path.isdir(self.tmp):
            shutil.rmtree(self.tmp)
        os.mkdir(self.tmp)
    def write(self, image, index):
        cv2.imwrite(os.path.join(self.tmp, f"{index:04}.png"), image)
    def save(self, fname):
        cmd = f"/usr/bin/ffmpeg -f image2 -i {self.tmp}/%04d.png -crf 0 -y {fname}"
        subprocess.call(cmd, shell=True)
        

In [ ]:
## Inspect and save a human trace ##

vista_reset()
stream = VideoStream()

for i in tqdm(range(100)):
    vista_step()
    
    # Render and save the display
    vis_img = display.render()
    stream.write(vis_img[:, :, ::-1], index=i)
    if car.done: 
        break

print("Saving trajectory of human following...")
stream.save("human_control.mp4")      

In [ ]:
mdl.lab3.play_video("human_control.mp4")

Check out the simulated VISTA environment. What do you notice about the environment, the agent, and the setup of the simulation engine? How could these aspects useful for training models?

### Define terminal states: crashing

Recall from Cartpole, our training episodes ended when the pole toppled, i.e., the agent crashed and failed. Similarly for training vehicle control policies in VISTA, we have to define what a ***crash*** means. We will define a crash as any time the car moves out of its lane or exceeds its maximum rotation. This will define the end of a training episode.

In [ ]:
## Define terminal states and crashing conditions ##

def check_out_of_lane(car):
    distance_from_center = np.abs(car.relative_state.x)
    road_width = car.trace.road_width 
    half_road_width = road_width / 2
    return distance_from_center > half_road_width

def check_exceed_max_rot(car):
    maximal_rotation = np.pi / 10.
    current_rotation = np.abs(car.relative_state.yaw)
    return current_rotation > maximal_rotation

def check_crash(car): 
    return check_out_of_lane(car) or check_exceed_max_rot(car) or car.done

### Use a random control policy

At this point, we have (1) an environment; (2) an agent, with a step function. Before we start learning a control policy for our vehicle agent, we start by testing out the behavior of the agent in the virtual world by providing it with a completely random control policy. Naturally we expect that the behavior will not be very robust! Let's take a look.

In [ ]:
## Behavior with random control policy ##

i = 0
num_crashes = 5
stream = VideoStream()

for _ in range(num_crashes):
    vista_reset()
    
    while not check_crash(car):

        # Sample a random curvature (between +/- 1/3), keep speed constant
        curvature = np.random.uniform(-1/3, 1/3)

        # Step the simulated car with the same action
        vista_step(curvature=curvature)

        # Render and save the display
        vis_img = display.render()
        stream.write(vis_img[:, :, ::-1], index=i)
        i += 1
    
    print(f"Car crashed on step {i}")
    for _ in range(5):
        stream.write(vis_img[:, :, ::-1], index=i)
        i += 1

print("Saving trajectory with random policy...")
stream.save("random_policy.mp4")

In [ ]:
mdl.lab3.play_video("random_policy.mp4")

## Data preprocessing

So, hopefully you saw that the random control policy was, indeed, not very robust. Yikes. Our overall goal in this lab is to build a self-driving agent using a neural network controller trained entirely in the simulator VISTA. This means that the data used to train and test the self-driving agent will be supplied by VISTA. As a step towards this, we will do some data preprocessing to make it easier for the network to learn from these visual data.

Previously we rendered the data with a display as a quick check that the environment was working properly. For training the agent, we will directly access the car's observations, extract Regions Of Interest (ROI) from those observations, crop them out, and use these crops as training data for our self-driving agent controller. Observe both the full observation and the extracted ROI.

In [ ]:
from google.colab.patches import cv2_imshow

# Directly access the raw sensor observations of the simulated car
vista_reset()
full_obs = car.observations[camera.name]
cv2_imshow(full_obs)

In [ ]:
## ROIs ##

# Crop a smaller region of interest (ROI). This is necessary because: 
#   1. The full observation will have distortions on the edge as the car deviates from the human
#   2. A smaller image of the environment will be easier for our model to learn from
region_of_interest = camera.camera_param.get_roi()
i1, j1, i2, j2 = region_of_interest
cropped_obs = full_obs[i1:i2, j1:j2]
cv2_imshow(cropped_obs)

We will group these steps into some helper functions that we can use during training: 
1. `preprocess`: takes a full observation as input and returns a preprocessed version. This can include whatever preprocessing steps you would like! For example, ROI extraction, cropping, augmentations, and so on. You are welcome to add and modify this function as you seek to optimize your self-driving agent!
2. `grab_and_preprocess`: grab the car's current observation (i.e., image view from its perspective), and then call the `preprocess` function on that observation.

In [ ]:
## Data preprocessing functions ##

def preprocess(full_obs):
    # Extract ROI
    i1, j1, i2, j2 = camera.camera_param.get_roi()
    obs = full_obs[i1:i2, j1:j2]
    
    # Rescale to [0, 1]
    obs = obs / 255.
    return obs

def grab_and_preprocess_obs(car):
    full_obs = car.observations[camera.name]
    obs = preprocess(full_obs)
    return obs

## Define the self-driving agent and learning algorithm

As before, we'll use a neural network to define our agent and output actions that it will take. Fixing the agent's driving speed, we will train this network to predict a curvature -- a continuous value -- that will relate to the car's turn radius. Specifically, define the model to output a prediction of a continuous distribution of curvature, defined by a mean $\mu$ and standard deviation $\sigma$. These parameters will define a Normal distribution over curvature.

What network architecture do you think would be especially well suited to the task of end-to-end control learning from RGB images? Since our observations are in the form of RGB images, we'll start with a convolutional network. Note that you will be tasked with completing a template CNN architecture for the self-driving car agent -- but you should certainly experiment beyond this template to try to optimize performance!

In [ ]:
### Define the self-driving agent ###

# input image size: CHW: 3,45,155
output_size = 2

def create_driving_model():
  model = torch.nn.Sequential(
      torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=2),
      torch.nn.Hardswish(),
      torch.nn.Conv2d(in_channels=32, out_channels=48, kernel_size=5, stride=2),
      torch.nn.Hardswish(),
      torch.nn.Conv2d(in_channels=48, out_channels=64, kernel_size=3, stride=2),
      torch.nn.Hardswish(),
      torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2),
      torch.nn.Hardswish(),
      torch.nn.Flatten(),
      torch.nn.Linear(512, 128),
      torch.nn.Hardswish(),
      torch.nn.Linear(128, output_size),
  )
  return model

driving_model = create_driving_model()
print(driving_model)

# # test
# x = torch.zeros((3, 3, 45, 155))
# driving_model(x)

Now we will define the learing algorithm which will be used to reinforce good behaviors of the agent and discourage bad behaviours. As with Cartpole, we will use a *policy gradient* method that aims to **maximize** the likelihood of actions that result in large rewards. However, there are some key differences. In Cartpole, the agent's action space was discrete: it could only move left or right. In driving, the agent's action space is continuous: the control network is outputting a curvature, which is a continuous variable. We will define a probability distribution, defined by a mean and variance, over this continuous action space to define the possible actions the self-driving agent can take.

You will define three functions that reflect these changes and form the core of the the learning algorithm:
1. `run_driving_model`: takes an input image, and outputs a prediction of a continuous distribution of curvature. This will take the form of a Normal distribuion and will be defined using TensorFlow probability's [`tfp.distributions.Normal`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/Normal) function, so the model's prediction will include both a mean and variance. Operates on an instance `driving_model` defined above.
2. `compute_driving_loss`: computes the loss for a prediction that is in the form of a continuous Normal distribution. Recall as in Cartpole, computing the loss involves multiplying the predicted log probabilities by the discounted rewards. Similar to `compute_loss` in Cartpole.

The `train_step` function to use the loss function to execute a training step will be the same as we used in Cartpole! This will have to be executed abov in order for the driving agent to train properly.

In [ ]:
## The self-driving learning algorithm ##

# hyperparameters
max_curvature = 1/8. 
max_std = 0.1 

def run_driving_model(image):
    # print(image.shape)
    image = np.copy(image)
    # print(image.shape)
    # Arguments:
    #   image: an input image
    # Returns:
    #   pred_dist: predicted distribution of control actions 
    single_image_input = image.ndim == 3  # missing 4th batch dimension
    if single_image_input:
        image = np.expand_dims(image, axis=0)

    # change NHWC to NCHW
    image = np.moveaxis(image, -1, 1)

    image = torch.from_numpy(image.astype(np.float32)).to(device)
    # get the prediction of the model given the current observation
    distribution = driving_model(image)

    mu, logsigma = torch.split(distribution, 1, dim=1)
    mu = max_curvature * torch.tanh(mu) # conversion
    sigma = max_std * torch.sigmoid(logsigma) + 0.005 # conversion
    
    # define the predicted distribution of curvature, given the predicted
    pred_dist = torch.distributions.normal.Normal(mu, sigma)
    return pred_dist


def compute_driving_loss(dist, actions, rewards):
    # Arguments:
    #   logits: network's predictions for actions to take
    #   actions: the actions the agent took in an episode
    #   rewards: the rewards the agent received in an episode
    # Returns:
    #   loss
    # complete the function call to compute the negative log probabilities
    actions = torch.from_numpy(actions.astype(np.float32)).to(device)
    rewards = torch.from_numpy(rewards.astype(np.float32)).to(device)
    neg_logprob = -1 * dist.log_prob(actions)
    # neg_logprob = '''TODO'''

    '''TODO: scale the negative log probability by the rewards.'''
    loss = torch.mean( neg_logprob * rewards ) # TODO
    # loss = tf.reduce_mean('''TODO''')
    return loss

# # test
# y = np.zeros((45, 155, 3))
# driving_model.to(device)
# dis = run_driving_model(y)
# a = dis.sample()[0, 0]
# print(a.device)




## Train the self-driving agent

We're now all set up to start training our RL algorithm and agent for autonomous driving!

We begin be initializing an opitimizer, environment, a new driving agent, and `Memory` buffer. This will be in the first code block. To restart training completely, you will need to re-run this cell to re-initiailize everything.

The second code block is the main training script. Here reinforcement learning episodes will be executed by agents in the VISTA environment. Since the self-driving agent starts out with literally zero knowledge of its environment, it can often take a long time to train and achieve stable behavior -- keep this in mind! For convenience, stopping and restarting the second cell will pick up training where you left off.

The training block will execute a policy in the environment until the agent crashes. When the agent crashes, the (state, action, reward) triplet `(s,a,r)` of the agent at the end of the episode will be saved into the `Memory` buffer, and then provided as input to the policy gradient loss function. This information will be used to execute optimization within the training step. Memory will be cleared, and we will then do it all over again!

Let's run the code block to train our self-driving agent. We will again visualize the evolution of the total reward as a function of training to get a sense of how the agent is learning. **You should reach a reward of at least 100 to get bare minimum stable behavior.**

In [ ]:
# some utils

### Reward function ###

# Helper function that normalizes an np.array x
def normalize(x):
    x -= np.mean(x)
    x /= np.std(x)
    return x.astype(np.float32)

# Compute normalized, discounted, cumulative rewards (i.e., return)
# Arguments:
#   rewards: reward at timesteps in episode
#   gamma: discounting factor
# Returns:
#   normalized discounted reward
def discount_rewards(rewards, gamma=0.95): 
    discounted_rewards = np.zeros_like(rewards)
    R = 0
    for t in reversed(range(0, len(rewards))):
        # update the total discounted reward
        R = R * gamma + rewards[t]
        discounted_rewards[t] = R
      
    return discounted_rewards

def normalized_discount_rewards(rewards, gamma=0.95):
    return normalize(discount_rewards(rewards, gamma))

### Agent Memory ###

class Memory:
    def __init__(self): 
        self.clear()

    # Resets/restarts the memory buffer
    def clear(self): 
        self.observations = []
        self.actions = []
        self.rewards = []

    # Add observations, actions, rewards to memory
    def add_to_memory(self, new_observation, new_action, new_reward): 
        self.observations.append(new_observation)
        '''TODO: update the list of actions with new action'''
        self.actions.append(new_action) # TODO
        # ['''TODO''']
        '''TODO: update the list of rewards with new reward'''
        self.rewards.append(new_reward) # TODO
        # ['''TODO''']

    # Get observations, actions, normalized_discount_rewards from an index array
    def get_from_memory(self, index):
        self.norm_disc_rew = normalized_discount_rewards(self.rewards)
        return np.array(self.observations)[index], np.array(self.actions)[index], np.array(self.norm_disc_rew)[index],

    def __len__(self):
        return len(self.actions)


In [ ]:
## Training parameters and initialization ##
## Re-run this cell to restart training from scratch ##

''' TODO: Learning rate and optimizer '''
# instantiate driving agent
vista_reset()
driving_model = create_driving_model()
driving_model.to(device)
# NOTE: the variable driving_model will be used in run_driving_model execution

learning_rate = 5e-4
# learning_rate = '''TODO'''
optimizer = torch.optim.Adam(params=driving_model.parameters(), lr=learning_rate)
# optimizer = '''TODO'''

# to track our progress
smoothed_reward = mdl.util.LossHistory(smoothing_factor=0.9)
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Rewards')

# instantiate Memory buffer
memory = Memory()

In [ ]:
## Driving training! Main training block. ##
## Note: stopping and restarting this cell will pick up training where you
#        left off. To restart training you need to rerun the cell above as 
#        well (to re-initialize the model and optimizer)

max_batch_size = 300
max_reward = float('-inf') # keep track of the maximum reward acheived during training
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists
for i_episode in range(500):

    plotter.plot(smoothed_reward.get())
    # Restart the environment
    vista_reset()
    memory.clear()
    observation = grab_and_preprocess_obs(car)

    # print("epidose: ", i_episode)
    # step = 0
    while True:
        # TODO: using the car's current observation compute the desired 
        #  action (curvature) distribution by feeding it into our 
        #  driving model (use the function you already built to do this!) '''
        curvature_dist = run_driving_model(observation)
        # curvature_dist = '''TODO'''
        
        # TODO: sample from the action *distribution* to decide how to step
        #   the car in the environment. You may want to check the documentation
        #   for tfp.distributions.Normal online. Remember that the sampled action
        #   should be a single scalar value after this step.
        curvature_action = curvature_dist.sample()[0,0].to('cpu').detach().numpy()
        # curvature_action = '''TODO'''
        
        # Step the simulated car with the same action
        vista_step(curvature=curvature_action)
        # print(step)
        # step += 1
               
        # TODO: Compute the reward for this iteration. You define 
        #   the reward function for this policy, start with something 
        #   simple - for example, give a reward of 1 if the car did not 
        #   crash and a reward of 0 if it did crash.
        reward = 1.0 if not check_crash(car) else 0.0
        #  reward = '''TODO'''
        
        # add to memory
        memory.add_to_memory(observation, curvature_action, np.float32(reward))

        observation = grab_and_preprocess_obs(car)
        
        # is the episode over? did you crash or do so well that you're done?
        if reward == 0.0:
            # determine total reward and keep a record of this
            total_reward = sum(memory.rewards)
            smoothed_reward.append(total_reward)
            
            # execute training step - remember we don't know anything about how the 
            #   agent is doing until it has crashed! if the training step is too large 
            #   we need to sample a mini-batch for this step.
            batch_size = min(len(memory), max_batch_size)
            i = np.random.choice(len(memory), batch_size, replace=False)
            # print(len(memory))
            batch_observations, batch_actions, batch_rewards = memory.get_from_memory(i)
            # print(batch_observations.shape)
            # print(batch_actions.shape)
            # print(batch_rewards.shape)
            pred_dist = run_driving_model(batch_observations)
            loss = compute_driving_loss(pred_dist, batch_actions, batch_rewards)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(driving_model.parameters(), max_norm=2)
            optimizer.step()
       
            # reset the memory
            memory.clear()
            break

## Evaluate the self-driving agent

Finally we can put our trained self-driving agent to the test! It will execute autonomous control, in VISTA, based on the learned controller. We will evaluate how well it does based on this distance the car travels without crashing. We await the result...

In [ ]:
## Evaluation block!##

i_step = 0
num_episodes = 5
num_reset = 5
stream = VideoStream()
for i_episode in range(num_episodes):
    
    # Restart the environment
    vista_reset()
    observation = grab_and_preprocess_obs(car)
    
    print("rolling out in env")
    episode_step = 0
    while not check_crash(car) and episode_step < 100:
        # using our observation, choose an action and take it in the environment
        curvature_dist = run_driving_model(observation)
        curvature = curvature_dist.sample()[0,0].to('cpu').detach().numpy()

        # Step the simulated car with the same action
        vista_step(curvature)
        observation = grab_and_preprocess_obs(car)

        vis_img = display.render()
        stream.write(vis_img[:, :, ::-1], index=i_step)
        i_step += 1
        episode_step += 1
        
    for _ in range(num_reset):
        stream.write(np.zeros_like(vis_img), index=i_step)
        i_step += 1
        
print(f"Average reward: {(i_step - (num_reset*num_episodes)) / num_episodes}")

print("Saving trajectory with trained policy...")
stream.save("trained_policy.mp4")

In [ ]:
mdl.lab3.play_video("trained_policy.mp4")

Congratulations for making it to this point and for training an autonomous vehicle control policy using deep neural networks and reinforcement learning! Now, with an eye towards the lab competition, think about what you can change -- about the controller model, your data, your learning algorithm... -- to improve performance even further. Below in 3.11 we have some suggestions to get you started. We hope to see your self-driving control policy put to the ***real*** test!